In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # or 'QtAgg' if you prefer
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import sys
sys.path.append('./')
from external.ADNet.lib.backbone import stackedHGNetV1
import torch
import os
import mediapipe as mp
import matplotlib
from Methods import *

# Manual Keypoint Detection

In [ ]:
img1 = cv2.imread("face_img.jpg")
img2 = cv2.imread("lion_img.jpg")

morph_img = Image_Morph(
    img1, img2,
    point_selection='MANUAL',
    alpha=0.5,
    output_image_path="outputs/morph.jpg",
    output_video_path="outputs/morph_loop.mp4"
)


### Display Average Morph (alpha=0.5)

In [3]:
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(morph_img, cv2.COLOR_BGR2RGB))
plt.title("Morphed Image")
plt.axis('off')
plt.show()

### Preview Morph Video

In [ ]:
preview_video('morph_loop.mp4')

# ADNET Keypoint Detection

In [ ]:
img1 = cv2.imread("face_img.jpg")
img2 = cv2.imread("lion_img.jpg")

morph_img2 = Image_Morph(
    img1, img2,
    point_selection='ADNET',
    alpha=0.5,
    output_image_path="outputs/morph2.jpg",
    output_video_path="outputs/morph_loop2.mp4",
    adnet_ckpt_dir="./train.pkl"
)


### Display Average Morph (alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(morph_img2, cv2.COLOR_BGR2RGB))
plt.title("Morphed Image")
plt.axis('off')
plt.show()

### Preview Morph Video

In [ ]:
preview_video('morph_loop2.mp4')

# Mediapipe Keypoint Detection

In [ ]:
# since mediapipe does not support non-human faces, lion image is not used
img1 = cv2.imread("./image_pairs/1/10_0_0_20161220222308131.jpg")
img2 = cv2.imread("./image_pairs/1/10_0_0_20170103200329407.jpg")

morph_img2 = Image_Morph(
    img1, img2,
    point_selection='MP',
    alpha=0.5,
    output_image_path="outputs/morph3.jpg",
    output_video_path="outputs/morph_loop3.mp4",
)

### Display Average Morph (alpha=0.5)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(morph_img2, cv2.COLOR_BGR2RGB))
plt.title("Morphed Image")
plt.axis('off')
plt.show()

### Preview Morph Video

In [ ]:
preview_video('morph_loop3.mp4')

### Check if morphing from A to B and B to A is symmetric

In [ ]:
run_symmetry_accuracy_test()